In [9]:
from tqdm import tqdm
import pickle
import torch
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
import torch.nn as nn
import torch.nn.functional as F
import os
import pandas as pd
import subprocess
import numpy as np
import torch
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
import os
from torch_geometric.loader import DataLoader
from pathlib import Path
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import DataLoader, Data
from torch_geometric.nn import GCNConv
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv
import random
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
from collections import Counter
from torch_geometric.data import Data, Dataset
import os
from sklearn.metrics import roc_auc_score, average_precision_score, balanced_accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
import matplotlib.pyplot as plt

from torch_geometric.nn import GATConv, global_mean_pool

In [10]:
%run ../Config.ipynb

In [11]:
config = Config()
patience = config.model_args["patience"]

#### BalancedBCEWithLogitsLoss

In [12]:
class BalancedBCEWithLogitsLoss(nn.Module):
    def __init__(self, pos_weight=None, reduction='mean'):
        super(BalancedBCEWithLogitsLoss, self).__init__()
        self.pos_weight = pos_weight
        self.reduction = reduction
        self.bce_loss = nn.BCEWithLogitsLoss(pos_weight=pos_weight, reduction=reduction)

    def forward(self, input, target):
        loss = self.bce_loss(input, target)
        return loss


In [13]:

# %run Dataforge.ipynb

class BipartiteData(Data):
    def __init__(self, edge_index=None, x_s=None, x_t=None, y=None, edge_attr=None):
        super().__init__()
        self.edge_index = edge_index
        self.x_s = x_s
        self.x_t = x_t
        self.y = y
        self.edge_attr = edge_attr  # Add edge_attr attribute
        #self.num_nodes = len(set(edge_index[0].tolist())) +  len(set(edge_index[1].tolist()))
        self.num_nodes = 10#(x_s.size(0) if x_s is not None else 0) + (x_t.size(0) if x_t is not None else 0)

    def __inc__(self, key, value, *args, **kwargs):
        if key == 'edge_index':
            return torch.tensor([[self.x_s.size(0)], [self.x_t.size(0)]])
        else:
            return super().__inc__(key, value, *args, **kwargs)


# To load the data back with the correct data types
with open(f'{config.data}/bipartite_data.pkl', 'rb') as file:
    dataset_list = pickle.load(file)

##### Filter data list
filtered_data_list_num_nodes = [data for data in dataset_list if data.num_nodes > 0]
##### Filter data list
filtered_data_list_descriptors = [data for data in filtered_data_list_num_nodes if
                                  data.x_s.shape[0] > 0 and data.x_t.shape[0] > 0]
filtered_data_list = filtered_data_list_descriptors[0:400]

#### Data info
label_distribution = dict(Counter([label.y.tolist() for label in filtered_data_list]))
amount_of_graphs_used_to_train = len(filtered_data_list)

## Dataset info

print(65 * "*")
# print("Nº Pdbs:", len(pdbs))
print("Nº BipartiteData objects:", len(dataset_list))
print("Nº BipartiteData objects filtered by num_nodes > 0: ", len(filtered_data_list_num_nodes))
print("Nº BipartiteData objects filtered by has descriptors > 0:", len(filtered_data_list_descriptors))
print("Nº BipartiteData objects for training", len(filtered_data_list))
print(65 * "*")

## Run Model

class BalancedBCEWithLogitsLoss(nn.Module):
    def __init__(self, pos_weight=None, reduction='mean'):
        super(BalancedBCEWithLogitsLoss, self).__init__()
        self.pos_weight = pos_weight
        self.reduction = reduction
        self.bce_loss = nn.BCEWithLogitsLoss(pos_weight=pos_weight, reduction=reduction)

    def forward(self, input, target):
        loss = self.bce_loss(input, target)
        return loss




*****************************************************************
Nº BipartiteData objects: 500
Nº BipartiteData objects filtered by num_nodes > 0:  436
Nº BipartiteData objects filtered by has descriptors > 0: 395
Nº BipartiteData objects for training 395
*****************************************************************


In [14]:
filtered_data_list[:10]

[BipartiteData(edge_index=[2, 62], x_s=[62, 7], x_t=[62, 7], y=0.0, edge_attr=[62], num_nodes=124, pdb='1w4o', pose=3),
 BipartiteData(edge_index=[2, 149], x_s=[149, 7], x_t=[149, 7], y=0.0, edge_attr=[149], num_nodes=298, pdb='1g2k', pose=2),
 BipartiteData(edge_index=[2, 71], x_s=[71, 7], x_t=[71, 7], y=0.0, edge_attr=[71], num_nodes=142, pdb='4djv', pose=3),
 BipartiteData(edge_index=[2, 86], x_s=[86, 7], x_t=[86, 7], y=0.0, edge_attr=[86], num_nodes=172, pdb='4eor', pose=6),
 BipartiteData(edge_index=[2, 82], x_s=[82, 7], x_t=[82, 7], y=0.0, edge_attr=[82], num_nodes=164, pdb='4qd6', pose=8),
 BipartiteData(edge_index=[2, 45], x_s=[45, 7], x_t=[45, 7], y=0.0, edge_attr=[45], num_nodes=90, pdb='3uev', pose=1),
 BipartiteData(edge_index=[2, 97], x_s=[97, 7], x_t=[97, 7], y=0.0, edge_attr=[97], num_nodes=194, pdb='3pww', pose=8),
 BipartiteData(edge_index=[2, 67], x_s=[67, 7], x_t=[67, 7], y=0.0, edge_attr=[67], num_nodes=134, pdb='4mme', pose=3),
 BipartiteData(edge_index=[2, 87], x_

In [44]:
train_loader = DataLoader(filtered_data_list[:10], batch_size=2, shuffle=False, follow_batch=['x_s', 'x_t'])

In [45]:
batch = next(iter(train_loader))

In [52]:
batch

BipartiteDataBatch(num_nodes=422, edge_index=[2, 211], x_s=[211, 7], x_s_batch=[211], x_s_ptr=[3], x_t=[211, 7], x_t_batch=[211], x_t_ptr=[3], y=[2], edge_attr=[211], pdb=[2], pose=[2], batch=[422], ptr=[3])

In [51]:
assert torch.all(batch.x_s_batch == batch.x_t_batch)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

#### GATModel

In [97]:
# Get today's date
from torch_geometric.nn import global_mean_pool

class GATModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size):
        super(GATModel, self).__init__()
        # Define the graph convolutional layers
        self.conv1 = GATConv(in_channels=input_dim, out_channels=hidden_dim)
        #self.conv2 = GATConv(in_channels=hidden_dim, out_channels=hidden_dim)

        self.fc = nn.Linear(hidden_dim, 1)  # Initialize fc layer as None
        
    def forward(self, data):
        x_s, x_t, edge_index, distances, x_t_batch = data.x_s, data.x_t, data.edge_index, data.edge_attr, data.x_t_batch
    
        # Print shapes of input tensors
        print("Shapes of input tensors:")
        print("x_s.shape:", x_s.shape)
        print("x_t.shape:", x_t.shape)
        print("edge_index.shape:", edge_index.shape)
        print("distances.shape:", distances.shape)
        print("x_t_batch.shape:", x_t_batch.shape)
        #print("x_t_batch:", x_t_batch)
        # Apply the graph convolutional layers
        x_new_t = self.conv1((x_s, x_t), edge_index, size=(x_s.size(0), x_t.size(0)))
        print("x_new_t.shape:", x_new_t.shape)
    
        x = torch.relu(x_new_t)
    
        x = global_mean_pool(x, x_t_batch) 
        print("x.shape after global_mean_pool:", x.shape)
        print('x after global_mean_pool', x, '\n')
        # Apply the linear layer
        x = self.fc(x) 
        print("x.shape after linear layer:", x.shape)
        print("x after linear layer:", x, '\n')
        
        return x.squeeze()

    def reset_parameters(self):
        for layer in [self.conv1, self.conv2]:
            if hasattr(layer, "reset_parameters"):
                layer.reset_parameters()


import time
import torch.optim as optim
# Initialize the model
model = GATModel(input_dim=7, hidden_dim=7, batch_size=config.model_args["batch_size"])
criterion = BalancedBCEWithLogitsLoss(pos_weight=torch.tensor(12))
optimizer = optim.Adam(model.parameters(), lr=config.model_args['lr'])
train_loader = DataLoader(filtered_data_list[:10], batch_size=2, shuffle=False, follow_batch=['x_s', 'x_t'])


In [98]:


#### Horário 
def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data)  # Perform a single forward pass.
         print("$data.y$", data.y)
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.


for epoch in range(1, 171):
    train()

Shapes of input tensors:
x_s.shape: torch.Size([211, 7])
x_t.shape: torch.Size([211, 7])
edge_index.shape: torch.Size([2, 211])
distances.shape: torch.Size([211])
x_t_batch.shape: torch.Size([211])
x_new_t.shape: torch.Size([211, 7])
x.shape after global_mean_pool: torch.Size([2, 7])
x after global_mean_pool tensor([[7.0943e-01, 4.8160e-01, 0.0000e+00, 4.4168e-01, 7.5323e-02, 2.2510e+00,
         1.0760e+00],
        [5.3542e-01, 3.6038e-01, 1.8519e-03, 9.2459e-01, 1.0872e-01, 1.4774e+00,
         7.5238e-01]], grad_fn=<DivBackward0>) 

x.shape after linear layer: torch.Size([2, 1])
x after linear layer: tensor([[-0.0494],
        [ 0.1017]], grad_fn=<AddmmBackward0>) 

$data.y$ tensor([0., 0.])
Shapes of input tensors:
x_s.shape: torch.Size([157, 7])
x_t.shape: torch.Size([157, 7])
edge_index.shape: torch.Size([2, 157])
distances.shape: torch.Size([157])
x_t_batch.shape: torch.Size([157])
x_new_t.shape: torch.Size([157, 7])
x.shape after global_mean_pool: torch.Size([2, 7])
x after gl

In [96]:
torch.tensor([[0.3191],
        [0.3076]]).squeeze()

tensor([0.3191, 0.3076])

In [78]:
from torch_geometric.utils import to_dense_batch

https://juandelacalle.medium.com/how-and-why-i-switched-from-the-roc-curve-to-the-precision-recall-curve-to-analyze-my-imbalanced-6171da91c6b8#:~:text=If%20the%20AUC%20equals%201,class%20far%20outweighs%20the%20other.